In [80]:
# ! pip install pandas numpy scikit-learn plotly matplotlib
# ! pip install nbformat
# ! pip install --upgrade nbformat
# ! pip install tensorflow[and-cuda]

In [81]:
# ! pip install pandas numpy scikit-learn plotly matplotlib
# ! pip install nbformat
# ! pip install --upgrade nbformat
# ! pip install tensorflow[and-cuda]

In [82]:
import pandas as pd
import numpy as np
import tensorflow as tf
from datetime import datetime as dt
import glob
from PIL import Image

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

from tensorflow.keras.layers import Dense, Input, Dropout, Rescaling, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Precision, Recall

from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

import warnings 
warnings.filterwarnings('ignore')

In [83]:
# Check if GPUs are available for training 
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [84]:
BATCH_SIZE=32

#### Train/test sets

In [85]:
# X_train, y_train
train_imgs = glob.glob(f'../CIFAKE/train/*/*')
X_train = []
y_train = []

for path, img_path in enumerate(train_imgs):
    class_ = img_path.split('/')[-2]

    # FAKE : 0 -  REAL : 1
    if class_=='FAKE': img_class = 0
    if class_=='REAL': img_class = 1

    # X_train to tensor
    img_pil = Image.open(img_path)
    img_np = np.asarray(img_pil)

    X_train.append(img_np)
    y_train.append(img_class)

X_train = np.asarray(X_train, dtype='uint8')
y_train = np.asarray(y_train)

# X_test, y_test
test_imgs = glob.glob(f'../CIFAKE/test/*/*')

X_test = []
y_test = []

for path, img_path in enumerate(test_imgs):
    class_ = img_path.split('/')[-2]

    # FAKE : 0 -  REAL : 1
    if class_=='FAKE': img_class = 0
    if class_=='REAL': img_class = 1

    # X_test to tensor
    img_pil = Image.open(img_path)
    img_np = np.asarray(img_pil)

    X_test.append(img_np)
    y_test.append(img_class)
    
X_test = np.asarray(X_train, dtype='uint8')
y_test = np.asarray(y_train)

In [86]:
print(f'X_train shape: {X_train.shape} | y_train shape: {y_train.shape}')
print(f'X_test shape: {X_test.shape} | y_test shape: {y_test.shape}')

X_train shape: (100000, 32, 32, 3) | y_train shape: (100000,)
X_test shape: (100000, 32, 32, 3) | y_test shape: (100000,)


#### Class balance

In [87]:
import glob
print('Class distribution: ')
print( f'Train REAL images: {len(glob.glob('../CIFAKE/train/REAL/*'))}'  )
print( f'Train FAKE images: {len(glob.glob('../CIFAKE/train/FAKE/*'))}'  )

print( f'Test REAL images: {len(glob.glob('../CIFAKE/test/REAL/*'))}'  )
print( f'Test FAKE images: {len(glob.glob('../CIFAKE/test/FAKE/*'))}'  )

Class distribution: 
Train REAL images: 50000
Train FAKE images: 50000
Test REAL images: 10000
Test FAKE images: 10000


In [88]:
# Show random image

#### Image scaling

In [89]:
# Image normalisation  1/255

#### Build model

In [90]:
# Build model
def CNN_model01():
    model = Sequential([
        Rescaling(1./255),
        Conv2D(32, 3, activation='relu'),
        MaxPooling2D(),
        Flatten(),

        Dense(24, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])

    model.compile( optimizer=Adam(learning_rate=0.01), 
                  loss=BinaryCrossentropy(),
                  metrics = ['accuracy', Precision(), Recall()],                
                  )

    model.build(input_shape=(None, 32, 32, 3))
    return model

In [91]:
# CNN_model01().summary()

#### Train model

In [92]:
N_EPOCHS = 30
# BATCH SIZE ?? # NOTE

In [97]:
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier
from datetime import datetime as dt


def train(model):
  time_start = dt.now()
  print(f'Start training, time: {time_start.time()}')

  history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=N_EPOCHS,
    verbose=1
  )

  print(f'Time elapsed: {dt.now() - time_start}')

  return history

# Train with GRID search
def train_gridSearch(model):

  keras_classifier = KerasClassifier(model=model)

  param_grid = dict(
      epochs=[25],
      # batch_size=[32],
  )

  grid = GridSearchCV(estimator=keras_classifier, param_grid=param_grid, cv=5, scoring='accuracy')

  start_t = dt.now()
  print(f'GridSearch start time: {start_t.hour}:{start_t.minute:02}:{start_t.second:02}')
  grid_result = grid.fit(X_train, y_train, validation_data=(X_test, y_test))

  print(f'GridSearch complete. Time elapsed: {dt.now()-start_t}')

  # Best results 
  print(f'Best parameters: {grid_result.best_params_}')
  print(f'Best result (R2): {grid_result.best_score_}')

  history = grid_result.best_estimator_.history_

  return history

In [98]:
history = train_gridSearch(CNN_model01())

GridSearch start time: 10:18:51
Epoch 1/25
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.5029 - loss: 0.7049 - precision_5: 0.5039 - recall_5: 0.3949 - val_accuracy: 0.5000 - val_loss: 0.6934 - val_precision_5: 0.5000 - val_recall_5: 1.0000
Epoch 2/25
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.4964 - loss: 0.6935 - precision_5: 0.4944 - recall_5: 0.4904 - val_accuracy: 0.5000 - val_loss: 0.6932 - val_precision_5: 0.5000 - val_recall_5: 1.0000
Epoch 3/25
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4963 - loss: 0.6934 - precision_5: 0.4975 - recall_5: 0.5366

KeyboardInterrupt: 

In [74]:
# from sklearn.model_selection import cross_val_score

# model = KerasClassifier(model=CNN_model01())

# fit_params = {
#     # TODO
# }

# k_fold_acc = cross_val_score(model, X_train, y_train, cv=10, scoring='accuracy', fit_params=fit_params)

2024-11-06 09:58:02.657669: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 276480000 exceeds 10% of free system memory.
2024-11-06 09:58:02.929120: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 276480000 exceeds 10% of free system memory.
I0000 00:00:1730883483.783462    2020 service.cc:146] XLA service 0x7fdfc40087f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730883483.783481    2020 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce GTX 1060 6GB, Compute Capability 6.1
2024-11-06 09:58:03.810438: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-11-06 09:58:04.076663: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


 171/2813 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5618 - loss: 0.7892 - precision_1: 0.5521 - recall_1: 0.7428

I0000 00:00:1730883485.712159    2020 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2813/2813 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - accuracy: 0.7053 - loss: 0.5733 - precision_1: 0.6799 - recall_1: 0.7886


2024-11-06 09:58:09.956792: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 30720000 exceeds 10% of free system memory.
2024-11-06 09:58:09.985838: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 30720000 exceeds 10% of free system memory.


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


2024-11-06 09:58:11.222949: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 276480000 exceeds 10% of free system memory.


2813/2813 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.4965 - loss: 0.7214 - precision_1: 0.4958 - recall_1: 0.4771
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 824us/step
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.7369 - loss: 0.5521 - precision_1: 0.7163 - recall_1: 0.7795
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 840us/step
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.7246 - loss: 0.5496 - precision_1: 0.7079 - recall_1: 0.7648
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 773us/step
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.6922 - loss: 0.5875 - precision_1: 0.6680 - recall_1: 0.7840
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 840us/step
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.7499 - loss: 0.5256 - precision_1: 0.7387 - recall_1: 0.7697
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 850us/step
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.7460 - loss: 0.5370 - precision_1: 0.7266 - recall_1: 0.7945
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 851us/step
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 6s 

In [75]:
k_fold_acc

array([0.7827, 0.5   , 0.8295, 0.787 , 0.7921, 0.7841, 0.8054, 0.7876,
       0.7786, 0.5   ])

#### Visualisation

In [76]:
cols = pd.DataFrame(history.history).columns
cols

NameError: name 'history' is not defined

In [65]:
hist = history.history

n = 5
train_col = cols[2]
val_col = f'val_{train_col}'

fig = make_subplots(rows=2, cols=2, subplot_titles=("Loss over epochs", f"{train_col} over epochs"), vertical_spacing=0.07)

# Loss
fig.add_trace( go.Scatter(x=list(range(len(hist['loss']))), y=hist['loss'], mode='lines+markers', name='Training Loss'), row=1, col=1 )
fig.add_trace( go.Scatter(x=list(range(len(hist['val_loss']))), y=hist['val_loss'], mode='lines+markers', name='Validation Loss'), row=1, col=1 )

# 
fig.add_trace(  go.Scatter(x=list(range(len(hist[train_col]))), y=hist[train_col],  mode='lines+markers', name=f'Training {train_col}'),  row=1, col=2 )
fig.add_trace( go.Scatter(x=list(range(len(hist[val_col]))), y=hist[val_col], mode='lines+markers', name=f'Validation {train_col}'), row=1, col=2)

#
train_col = cols[3]
val_col = f'val_{train_col}'
fig.add_trace(  go.Scatter(x=list(range(len(hist[train_col]))), y=hist[train_col],  mode='lines+markers', name=f'Training {train_col}'),  row=2, col=1 )
fig.add_trace( go.Scatter(x=list(range(len(hist[val_col]))), y=hist[val_col], mode='lines+markers', name=f'Validation {train_col}'), row=2, col=1)

train_col = 'accuracy'
val_col = f'val_{train_col}'
fig.add_trace(  go.Scatter(x=list(range(len(hist[train_col]))), y=hist[train_col],  mode='lines+markers', name=f'Training {train_col}'),  row=2, col=2 )
fig.add_trace( go.Scatter(x=list(range(len(hist[val_col]))), y=hist[val_col], mode='lines+markers', name=f'Validation {train_col}'), row=2, col=2)


fig.update_xaxes(title_text="Epochs", row=1, col=1)
fig.update_yaxes(title_text="Loss", row=1, col=1)
fig.update_yaxes(title_text=f"{train_col}", row=1, col=2)
fig.update_yaxes(title_text=f"{train_col}", row=2, col=1)
fig.update_yaxes(title_text=f"{train_col}", row=2, col=2)

fig.update_layout(
    # title_text="Training and validation metrics over epochs",
    showlegend=True,
    margin=dict(l=10, r=10, b=10, t=30),
    width=1400, height=800
)

for annotation in fig['layout']['annotations']:
    annotation['y'] = annotation['y'] + 0.002

fig

In [66]:
# Confusion matrix
# conf_matrix = confusion_matrix(y_test, y_pred_classes)

#### Class activation map